In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from src.helpers.serialization import df_read_json
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import numpy as np

In [4]:
amazon_df = pd.read_csv('../../data/amazon_sorted/most_populair_products.csv')
amazon_df = amazon_df.drop(columns='index').dropna().reset_index()

product_ids = amazon_df['product_id'].unique()

In [7]:

sia = SentimentIntensityAnalyzer()

def measure_deviation(summaries: pd.DataFrame) -> str:
    avg_sent_inputs = list()
    for index, product_id in enumerate(product_ids):
        reviews = amazon_df[amazon_df['product_id'] == product_id]['review_body']
        summary = summaries.iloc[index]['text']
        sents = reviews.apply(sia.polarity_scores)
        sents = sents.apply(lambda x: x.get('compound'))
        avg_sent_input = sents.mean()
        sent_sum = sia.polarity_scores(summary).get('compound')
        avg_sent_inputs.append(abs(avg_sent_input - sent_sum))
    return str(sum(avg_sent_inputs) / len(avg_sent_inputs))

topsum_summaries = df_read_json('../../results/topsum_summaries.json')
print(measure_deviation(topsum_summaries))

0.9536928373712051


In [8]:
distilbart_summaries = df_read_json('../../data/distilbart_on_amazon_summaries.json')
print(measure_deviation(distilbart_summaries))

0.5330465044047651
